# Sheet 1 - Practical Exercise
The datasets we will be using throughout the practical exercises of this course are subsets of the *FIFA 20 complete player dataset* from kaggle.com, which is extracted from the latest edition of FIFA, a football simulation game. In this dataset the player's attributes like personal data (e.g. Nationality, Club, Age, Value, …) and performances (e.g. Agression, Agility, Overall, Potential, …) are collected. (See [FIFA 20 complete player dataset](https://www.kaggle.com/stefanoleone992/fifa-20-complete-player-dataset?select=players_20.csv) for details.)

For the individual tasks we provide you with preprocessed data.

Before we start with the actual exercises make sure that you have imported all packages that you need, and loaded the dataset.

In [200]:
# Add your imports and load your data here
import numpy as np
import pandas as pd
import timeit

## check the filepath is correct or not. For mycase the csv file is in the same directory as the ipynb file
fifa_df = pd.read_csv('fifa_20.csv')

## Exercise 1


First create a smaller Dataframe containing only the columns relevant for the coming exercises. These columns are:
- ``short_name``
- ``age``
- ``club``
- ``overall``
- ``potential``
- ``value_eur``
- ``wage_eur``
- ``team_position``

In [201]:
# add code creating a Dataframe with a subset of all available columns
players = fifa_df.loc[ : , 
                [
                 'short_name', 
                 'age', 
                 'club', 
                 'overall', 
                 'potential', 
                 'value_eur', 
                 'wage_eur', 
                 'team_position'
                ] 
           ]



Next write a scaler to scale the values of a column to an interval between a new minimum and maximum value.  
It should use the following signature:

_Hint: Do **not** use a loop over all entries of the column! What other ways are there to manipulate columnar data?_

In [202]:
def my_scaler(column,min_val,max_val):
    # Add your code here
    
    #assignment
    max_new = max_val
    min_new = min_val
    max_old = players['value_eur'].max()
    min_old = players['value_eur'].min()
    
    #calculation
    scaled_column = ( ( (max_new - min_new) / (max_old - min_old) ) * (column - min_old)  ) + min_new
    
    return scaled_column

To achieve this your scaler can implement the formula:  
$$\frac{max_{new}-min_{new}}{max_{old}-min_{old}}\cdot (v-min_{old})+min_{new}$$
where $max_{new}$ and $min_{new}$ are the new boundaries of the values, $max_{old}$ and $min_{old}$ are the old boundaries and $v$ is a value from the column that will be scaled.

* Now use your scaler on the column ``value_eur`` and scale it to an interval between $0$ and $100$.  
* Then take the new values for the ``value_eur`` und turn them into discrete values. If a player is below $25$ he should be categorized as ``low``, between $25$ and $50$ he should be categorized as ``low_average`` then between $50$ and $75$ as ``high_average`` and finally any that are above $75$ should be categorized as ``high``.  
* Finally create a histogram to show the distribution of the players' value.

In [203]:
#%%timeit

# Add code to use your scaler on the value_eur-column
intervalEndPoint1 = 0
intervalEndPoint2 = 100
targetIndex = 'value_eur'

# players['value_eur'] =  players[targetIndex].apply(lambda row: my_scaler(row, intervalEndPoint1, intervalEndPoint2))

players['value_eur'] = my_scaler( players.loc[ : , targetIndex], intervalEndPoint1, intervalEndPoint2  )

players.head(10)
# players.dtypes


,short_name,age,club,overall,potential,value_eur,wage_eur,team_position
0,L. Messi,32,FC Barcelona,94,94,90.521327,565000,RW
1,Cristiano Ronaldo,34,Juventus,93,93,55.450237,405000,LW
2,Neymar Jr,27,Paris Saint-Germain,92,92,100.000000,290000,CAM
3,J. Oblak,26,Atlético Madrid,91,93,73.459716,125000,GK
4,E. Hazard,28,Real Madrid,91,91,85.308057,470000,LW
5,K. De Bruyne,28,Manchester City,91,91,85.308057,370000,RCM
6,M. ter Stegen,27,FC Barcelona,90,93,63.981043,250000,GK
7,V. van Dijk,27,Liverpool,90,91,73.933649,200000,LCB
8,L. Modrić,33,Real Madrid,90,90,42.654028,340000,RCM
9,M. Salah,27,Liverpool,90,90,76.303318,240000,RW


In [204]:
# Add code to turn the new values of the value_eur column into the given discrete values

#using ternary
conditionDf = players['value_eur']
players['value_eur'] = np.where(
          (conditionDf < 25), 'low',
            (
                np.where(
                           ( conditionDf > 25) & (conditionDf < 50), 'low_average',
                              np.where( (conditionDf > 50) & (conditionDf < 75), 'high_average', 'high'
                            )
                        )
            )
        )

players

,short_name,age,club,overall,potential,value_eur,wage_eur,team_position
0,L. Messi,32,FC Barcelona,94,94,high,565000,RW
1,Cristiano Ronaldo,34,Juventus,93,93,high_average,405000,LW
2,Neymar Jr,27,Paris Saint-Germain,92,92,high,290000,CAM
3,J. Oblak,26,Atlético Madrid,91,93,high_average,125000,GK
4,E. Hazard,28,Real Madrid,91,91,high,470000,LW
...,...,...,...,...,...,...,...,...
18273,Shao Shuai,22,Beijing Renhe FC,48,56,low,2000,RES
18274,Xiao Mingjie,22,Shanghai SIPG FC,48,56,low,2000,SUB
18275,Zhang Wei,19,Hebei China Fortune FC,48,56,low,1000,SUB
18276,Wang Haijian,18,Shanghai Greenland Shenhua FC,48,54,low,1000,SUB


In [205]:
# Add code to create a histogram
import plotly.express as px

fig = px.histogram(players, x="value_eur")
fig.show()


## Exercise 2

Create a new column called ``leftover-potential`` which shows the difference between the value of ``overall`` and the value of ``potential``, to show how much more a given player can still grow.

In [206]:
# Add code to create the 'leftover-potential'-column

players["leftover-potential"] = players["overall"] - players["potential"]


Next create a histogram that shows how often how much potential is still left over.

In [151]:
# Add code to create the histogram

fig = px.histogram(players, x="leftover-potential")
fig.show()


## Exercise 3

Group the players by the club they are playing for. Then calculate the average of the ``overall`` column for each team.  

In [207]:
# Add code to group players and calculate the average 'overall' value of each club
playersGroupBy = players.groupby('club', as_index=False)['overall'].mean()

Then print the five Teams with the highest as well as the five teams with the lowest average ``overall`` value, so we can see which teams are the strongest and which teams are the weakest.

In [208]:
# Add code to select the strongest and weakest teams
strongestTeams = playersGroupBy.nlargest(5,'overall')
weekestTeams = playersGroupBy.nsmallest(5,'overall')

theTeams = strongestTeams.append(weekestTeams, ignore_index=True)

theTeams
    

,club,overall
0,FC Bayern München,81.304348
1,Real Madrid,80.121212
2,Juventus,80.060606
3,Uruguay,78.608696
4,FC Barcelona,78.363636
5,UCD AFC,53.428571
6,Finn Harps,55.280000
7,Crewe Alexandra,55.800000
8,Waterford FC,55.850000
9,Bolton Wanderers,56.086957


Next determine the sum of the ``wages`` of each team. Print the five teams with the highest total ``wages`` and the five teams with the lowest total ``wages``, but only those that spend more than 0€ in total.

In [209]:
# Add code to calculate the total wages and print the highest and lowest spenders

#filter the data where wage_eur is more than Zero :)
filteredClubs = players.loc[players.loc[:, 'wage_eur'] > 0]

playersGroupByWages = filteredClubs.groupby('club', as_index=False)['wage_eur'].sum()

highestPaidClubs = playersGroupByWages.nlargest(5,'wage_eur')
lowestPaidClubs = playersGroupByWages.nsmallest(5,'wage_eur')

results = pd.concat([highestPaidClubs, lowestPaidClubs], ignore_index=True, sort=False)

results

,club,wage_eur
0,Real Madrid,5354000
1,FC Barcelona,4950000
2,Manchester City,3984000
3,Juventus,3750000
4,Manchester United,2874000
5,GIF Sundsvall,1000
6,Seongnam FC,1000
7,US Orléans Loiret Football,1000
8,Śląsk Wrocław,1000
9,Shonan Bellmare,4000


Do the greatest spenders and strongest teams match up? What about the lowest spenders and the weakest teams?

For the greatest spenders and the storgest team doesn't match up. 
Because for example Bayern got the highest overall which is 81.304348. But highest paid club is Real Madrid. 
And also in the top five highest paid teams Bayern is not included. So we can not say that a club is stronger than 
other clubs if it pays more wages. 

And the case is also same for the lowest spenders and the weakest teams. It also doesn't match.



